In [ ]:
import math
import pandas as pd
import scipy.constants as const
import scipy.integrate as integrate
from scipy.interpolate import interp1d
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Constants
h = const.h  # Planck constant
c = const.c  # Speed of light
k = const.k  # Boltzmann constant
T = 5778  # Temperature of the Sun's surface in Kelvin

In [ ]:

# Constants for CMC simulation
consts = {
    'N': 7,
    'ω': [1.13, 0.96, 0.89, 1.09, 0.98, 1.04, 0.86],
    'γ': [0.09780352, 0.2154919, 0.2159708, 0.17640289, 0.15794933, 0.09051265, 0.20707075],
    'γφ': [0.333, 0.292, 0.364, 0.228, 0.317, 0.251, 0.375],
    'Γ': [0.064, 0.045, 0.058, 0.077, 0.051, 0.068, 0.044],
    'J': [[0.45, 0.03, 0.38, 0.47, 0.29, 0.08, 0.41],
          [0.10, 0.40, 0.48, 0.11, 0.30, 0.17, 0.15],
          [0.01, 0.43, 0.19, 0.36, 0.05, 0.14, 0.22],
          [0.35, 0.33, 0.27, 0.25, 0.49, 0.44, 0.42],
          [0.39, 0.26, 0.18, 0.50, 0.07, 0.20, 0.16],
          [0.48, 0.37, 0.45, 0.25, 0.31, 0.17, 0.35],
          [0.02, 0.19, 0.34, 0.28, 0.46, 0.40, 0.21]]
}

# Pauli matrices for CMC simulation
pauli = {
    '+': np.array([[0, 1], [0, 0]]),
    '-': np.array([[0, 0], [1, 0]]),
    'z': np.array([[1, 0], [0, -1]])
}

# CMC simulation functions
def sigma(i, op):
    return np.kron(np.eye(2**i), np.kron(pauli[op], np.eye(2**(consts['N']-i-1))))

def H():
    N = consts['N']
    ω = consts['ω']
    J = consts['J']
    H_val = np.zeros((2**N, 2**N))
    for i in range(N):
        H_val += ω[i] * sigma(i, 'z')
        for j in range(i+1, N):
            H_val += J[i][j] * (sigma(i, '+') @ sigma(j, '-') + sigma(i, '-') @ sigma(j, '+'))
    return H_val

def anticomm(A, B, C):
    return np.dot(A, np.dot(B, C)) + np.dot(B, np.dot(A, C))

def decay(rho, i):
    return consts['γ'][i] * (sigma(i, '-') @ rho @ sigma(i, '+') - 0.5 * anticomm(sigma(i, '+'), sigma(i, '-'), rho))

def dephase(rho, i):
    return consts['γφ'][i] * (sigma(i, 'z') @ rho @ sigma(i, 'z') - rho)

def emit(rho, i):
    return consts['Γ'][i] * (sigma(i, '-') @ rho @ sigma(i, '+'))

def rhs(t, rho_vec):
    rho = rho_vec.reshape((2**consts['N'], 2**consts['N']))
    dρdt = -1j * np.dot(H(), rho) + 1j * np.dot(rho, H())
    for i in range(consts['N']):
        dρdt += decay(rho, i) + dephase(rho, i) + emit(rho, i)
    return dρdt.flatten()

def simulate_CMC(G_initial):
    rho0 = np.eye(2**consts['N']) / 2**consts['N']
    times = np.linspace(0, 10, 100)
    result = solve_ivp(rhs, (0, 10), rho0.flatten(), t_eval=times)
    P_excited = np.kron([[0, 0], [0, 1]], np.eye(2**(consts['N']-1)))
    G_final = G_initial * np.trace(np.dot(P_excited, result.y[:, -1].reshape((2**consts['N'], 2**consts['N']))))
    return G_final

# Apply CMC simulation to solar simulation results

df_results = pd.read_csv('output.csv')
df_results['Final_Exciton_Rate_After_CMC'] = df_results['exciton_generation_rate'].apply(simulate_CMC)
df_results.to_csv('output.csv')


In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))  # Adjust the figure size if needed
plt.plot(df_results.index, df_results['Final_Exciton_Rate_After_CMC'], marker='o', linestyle='-')

# Set labels and title
plt.xlabel('Hour')
plt.ylabel('Final Exciton Rate After CMC')
plt.title('Final Exciton Rate After CMC vs Hour')

plt.savefig('Final_Exciton_Rate_After_CMC.png')
plt.show()